<a href="https://colab.research.google.com/github/seirah-yang/BootCamp/blob/main/%EC%8A%A4%EC%BF%BC%ED%8A%B8_%EC%8B%A4%EC%8A%B50904.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu


In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.1 MB/s eta 0:00:00


In [3]:
!pip install opencv-python numpy moviepy Pillow

In [6]:
!pip install yt-dlp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.0/176.0 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 67.4 MB/s eta 0:00:00


In [7]:
import os
import cv2
from yt_dlp import YoutubeDL
from ultralytics import YOLO

# 1) 유튜브 영상 다운로드(가장 좋은 mp4)
url = "https://youtube.com/shorts/GAkdsDbmZs8?si=HY3MmqYraE-HPRIf"
ydl_opts = {
    "format": "best[ext=mp4]/best",   # mp4 우선
    "outtmpl": "input_video.%(ext)s",
    "noplaylist": True
}
with YoutubeDL(ydl_opts) as ydl:
    info = ydl.extract_info(url, download=True)
    input_video = ydl.prepare_filename(info)  # 예: input_video.mp4

assert os.path.isfile(input_video), f"다운로드 실패: {input_video}"

# 2) 모델 로드 (Pose)
model = YOLO("yolov8n-pose.pt")

# 3) 비디오 열기
cap = cv2.VideoCapture(input_video)
if not cap.isOpened():
    raise RuntimeError(f"비디오를 열 수 없습니다: {input_video}")

# 4) 메타정보 + 안전가드
fps = cap.get(cv2.CAP_PROP_FPS)
if not fps or fps <= 1e-6:
    fps = 30.0  # 백업값

frame_width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)  or 0)
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) or 0)
if frame_width <= 0 or frame_height <= 0:
    # 한 번 읽어서 크기를 보정
    ok, test_frame = cap.read()
    if not ok:
        cap.release()
        raise RuntimeError("첫 프레임을 읽을 수 없습니다.")
    frame_height, frame_width = test_frame.shape[:2]
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # 되돌리기

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
total_duration = (total_frames / fps) if total_frames > 0 else 0.0
print(f"FPS={fps:.3f}, size={frame_width}x{frame_height}, frames={total_frames}, dur≈{total_duration:.2f}s")

# 5) 비디오 저장 설정 (mp4로 저장)
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # 또는 "XVID"(.avi)
out_path = "output_pose.mp4"
out = cv2.VideoWriter(out_path, fourcc, fps, (frame_width, frame_height))

# 6) 프레임 루프 + YOLO Pose 추론
while True:
    ok, frame = cap.read()
    if not ok:
        break

    # YOLO Pose inference (반환: 키포인트 포함)
    results = model(frame, verbose=False)
    # 시각화(annotated_frame)
    annotated = results[0].plot()  # bbox+keypoints 그려진 프레임

    out.write(annotated)

cap.release()
out.release()
print(f"저장 완료: {out_path}")


[youtube] Extracting URL: https://youtube.com/shorts/GAkdsDbmZs8?si=HY3MmqYraE-HPRIf
[youtube] GAkdsDbmZs8: Downloading webpage
[youtube] GAkdsDbmZs8: Downloading tv simply player API JSON
[youtube] GAkdsDbmZs8: Downloading tv client config
[youtube] GAkdsDbmZs8: Downloading player 79e70f61-main
[youtube] GAkdsDbmZs8: Downloading tv player API JSON
[info] GAkdsDbmZs8: Downloading 1 format(s): 18
[download] Sleeping 3.00 seconds as required by the site...
[download] Destination: input_video.mp4
[download] 100% of    1.04MiB in 00:00:00 at 2.91MiB/s   
FPS=29.970, size=360x640, frames=439, dur≈14.65s
저장 완료: output_pose.mp4


In [8]:
# !pip install -q yt-dlp ultralytics opencv-python moviepy Pillow
# !pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

import os
import cv2
import numpy as np
from yt_dlp import YoutubeDL
from ultralytics import YOLO

# ============ 0) 유튜브 → mp4 다운로드 ============
url = "https://youtube.com/shorts/GAkdsDbmZs8?si=HY3MmqYraE-HPRIf"
ydl_opts = {
    "format": "best[ext=mp4]/best",
    "outtmpl": "input_video.%(ext)s",
    "noplaylist": True
}
with YoutubeDL(ydl_opts) as ydl:
    info = ydl.extract_info(url, download=True)
    input_video = ydl.prepare_filename(info)  # 예: input_video.mp4

assert os.path.isfile(input_video), f"다운로드 실패: {input_video}"

# ============ 1) 모델 로드 ============
model = YOLO("yolov8n-pose.pt")

# ============ 2) 비디오 열기 ============
cap = cv2.VideoCapture(input_video)
if not cap.isOpened():
    raise RuntimeError(f"비디오를 열 수 없습니다: {input_video}")

fps = cap.get(cv2.CAP_PROP_FPS)
if not fps or fps <= 1e-6:
    fps = 30.0  # 백업값

frame_width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)  or 0)
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) or 0)

# 필요 시 한 프레임 읽어 크기 보정
if frame_width <= 0 or frame_height <= 0:
    ok, test_frame = cap.read()
    if not ok:
        cap.release()
        raise RuntimeError("첫 프레임을 읽을 수 없습니다.")
    frame_height, frame_width = test_frame.shape[:2]
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# ============ 3) 저장 준비 ============
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out_path = "output_pose_squat.mp4"
out = cv2.VideoWriter(out_path, fourcc, fps, (frame_width, frame_height))

# ============ 4) 각도 계산 함수 ============
def calculate_angle(a, b, c):
    """3개의 (x,y) 포인트를 받아 각도를 계산"""
    a = np.array(a[:2], dtype=float)
    b = np.array(b[:2], dtype=float)
    c = np.array(c[:2], dtype=float)

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360.0 - angle
    return angle

# ============ 5) 카운팅/판정 상태 변수 ============
count = 0
in_descending_phase = False
in_ascending_phase = False

bad_posture_detected = False
min_knee_angle_descending = None
max_knee_angle_ascending = None
max_hip_angle_ascending = None

confidence_threshold = 0.5
error_frames = []
frame_count = 0

# ============ 6) 프레임 루프 ============
while True:
    ok, frame = cap.read()
    if not ok:
        break

    # 기본값
    knee_angle_right = hip_angle_right = 0.0
    knee_angle_left  = hip_angle_left  = 0.0

    # YOLO pose
    results = model(frame, verbose=False)
    r = results[0]

    # 감지된 사람이 없으면 다음 프레임
    if (not hasattr(r, "keypoints")) or (r.keypoints is None) or (r.keypoints.xy is None) or (len(r.keypoints.xy) == 0):
        # 카운트만 표시하고 저장
        cv2.putText(frame, f'Count: {count}', (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        out.write(frame)
        frame_count += 1
        continue

    # 여러 명이면 가장 큰 bbox의 사람 1명 선택
    person_idx = 0
    if r.boxes is not None and len(r.boxes) > 0:
        areas = []
        for i, b in enumerate(r.boxes.xyxy.cpu().numpy()):
            x1, y1, x2, y2 = b
            areas.append((i, (x2-x1)*(y2-y1)))
        person_idx = max(areas, key=lambda t: t[1])[0]

    # keypoints: (n,17,2), conf: (n,17)
    kpts_xy = r.keypoints.xy.cpu().numpy()[person_idx]     # (17,2)
    kpts_cf = None
    if hasattr(r.keypoints, "conf") and (r.keypoints.conf is not None):
        kpts_cf = r.keypoints.conf.cpu().numpy()[person_idx]  # (17,)

    # COCO 17-keypoint 인덱스 매핑
    #  5: L-shoulder,  6: R-shoulder
    # 11: L-hip,      12: R-hip
    # 13: L-knee,     14: R-knee
    # 15: L-ankle,    16: R-ankle
    left_shoulder  = (*kpts_xy[5],  kpts_cf[5]  if kpts_cf is not None else 1.0)
    right_shoulder = (*kpts_xy[6],  kpts_cf[6]  if kpts_cf is not None else 1.0)
    left_hip       = (*kpts_xy[11], kpts_cf[11] if kpts_cf is not None else 1.0)
    right_hip      = (*kpts_xy[12], kpts_cf[12] if kpts_cf is not None else 1.0)
    left_knee      = (*kpts_xy[13], kpts_cf[13] if kpts_cf is not None else 1.0)
    right_knee     = (*kpts_xy[14], kpts_cf[14] if kpts_cf is not None else 1.0)
    left_ankle     = (*kpts_xy[15], kpts_cf[15] if kpts_cf is not None else 1.0)
    right_ankle    = (*kpts_xy[16], kpts_cf[16] if kpts_cf is not None else 1.0)

    # 신뢰도 체크
    def ok4(*pts):
        return all(p[2] >= confidence_threshold for p in pts)

    if ok4(right_hip, right_knee, right_ankle, right_shoulder) and ok4(left_hip, left_knee, left_ankle, left_shoulder):
        # 오른쪽 각도
        knee_angle_right = calculate_angle(right_hip, right_knee, right_ankle)
        hip_angle_right  = calculate_angle(right_shoulder, right_hip, right_knee)
        # 왼쪽 각도
        knee_angle_left  = calculate_angle(left_hip, left_knee, left_ankle)
        hip_angle_left   = calculate_angle(left_shoulder, left_hip, left_knee)
    else:
        # 신뢰도 낮으면 계산 생략(0 유지) 혹은 한쪽만 계산하려면 분기 추가 가능
        pass

    # (x,y) 정수 좌표
    def to_xy(p): return (int(round(p[0])), int(round(p[1])))

    sr, hr, kr, ar = map(to_xy, [right_shoulder, right_hip, right_knee, right_ankle])
    sl, hl, kl, al = map(to_xy, [left_shoulder,  left_hip,  left_knee,  left_ankle])

    # 보조 라인 그리기
    YELLOW = (0, 255, 255)
    ORANGE = (0, 165, 255)
    cv2.line(frame, sr, hr, YELLOW, 2); cv2.line(frame, sl, hl, YELLOW, 2)
    cv2.line(frame, hr, kr, ORANGE, 2); cv2.line(frame, hl, kl, ORANGE, 2)
    cv2.line(frame, kr, ar, ORANGE, 2); cv2.line(frame, kl, al, ORANGE, 2)

    # 각도 텍스트
    cv2.putText(frame, f'Hip Angle R: {hip_angle_right:.1f}',  (10, 80),  cv2.FONT_HERSHEY_SIMPLEX, 0.6, YELLOW, 2, cv2.LINE_AA)
    cv2.putText(frame, f'Knee Angle R:{knee_angle_right:.1f}', (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.6, YELLOW, 2, cv2.LINE_AA)
    cv2.putText(frame, f'Hip Angle L: {hip_angle_left:.1f}',   (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.6, ORANGE, 2, cv2.LINE_AA)
    cv2.putText(frame, f'Knee Angle L:{knee_angle_left:.1f}',  (10, 140), cv2.FONT_HERSHEY_SIMPLEX, 0.6, ORANGE, 2, cv2.LINE_AA)

    # 좌우 중 min/max
    min_knee_angle = min(knee_angle_right, knee_angle_left)
    max_knee_angle = max(knee_angle_right, knee_angle_left)
    min_hip_angle  = min(hip_angle_right,  hip_angle_left)
    max_hip_angle  = max(hip_angle_right,  hip_angle_left)

    # 올라오는 구간의 힙 최대값 갱신
    if in_ascending_phase:
        max_hip_angle_ascending = max(max_hip_angle_ascending or 0.0, max_hip_angle)

    # -------- 스쿼트 상태머신 --------
    # 내려가는 단계: 무릎이 110° 미만
    if min_knee_angle < 110:
        if not in_descending_phase:
            in_descending_phase = True
            in_ascending_phase = False
            min_knee_angle_descending = min_knee_angle
            max_knee_angle_ascending = None
            max_hip_angle_ascending = None
        else:
            min_knee_angle_descending = min(min_knee_angle_descending, min_knee_angle)

    # 올라오는 단계: 무릎이 거의 펴짐(>170°)
    elif max_knee_angle > 170:
        if in_descending_phase:
            in_ascending_phase = True
            # 초기화(최대값은 이후 프레임에서 갱신될 수 있음)
            max_knee_angle_ascending = max_knee_angle
            max_hip_angle_ascending  = max_hip_angle
        else:
            max_knee_angle_ascending = max(max_knee_angle_ascending or 0.0, max_knee_angle)
            max_hip_angle_ascending  = max(max_hip_angle_ascending  or 0.0, max_hip_angle)

        # 1회 카운트 조건: 무릎>170 & 엉덩이>170
        if in_ascending_phase and (max_knee_angle_ascending and max_knee_angle_ascending > 170) and (max_hip_angle_ascending and max_hip_angle_ascending > 170):
            count += 1
            in_descending_phase = False
            in_ascending_phase = False
            min_knee_angle_descending = None
            max_knee_angle_ascending = None
            max_hip_angle_ascending = None

    # 충분히 내려가지 않음(내림 중 최소 무릎 각도가 120° 이상)
    elif in_descending_phase and min_knee_angle_descending is not None:
        if min_knee_angle_descending > 120:
            bad_posture_detected = True
            error_frames.append(frame_count)
            cv2.putText(frame, '자세불량: 충분히 내려가지 않음', (10, 180), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2, cv2.LINE_AA)
            in_descending_phase = False
            min_knee_angle_descending = None
            max_knee_angle_ascending = None
            max_hip_angle_ascending = None

    # 충분히 펴지지 않음(올림 중 힙 최대각 < 170°)
    elif in_ascending_phase and max_hip_angle_ascending is not None:
        if max_hip_angle_ascending < 170:
            bad_posture_detected = True
            error_frames.append(frame_count)
            cv2.putText(frame, '자세불량: 충분히 펴지지 않음', (10, 180), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2, cv2.LINE_AA)
            in_ascending_phase = False
            min_knee_angle_descending = None
            max_knee_angle_ascending = None
            max_hip_angle_ascending = None

    # 카운트 오버레이
    cv2.putText(frame, f'Count: {count}', (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2, cv2.LINE_AA)

    # 저장
    out.write(frame)
    frame_count += 1

# ============ 7) 종료 처리 ============
cap.release()
out.release()

print(f"최종 카운트: {count}")
print(f"오류 프레임 목록: {error_frames}")
print(f"저장 완료: {out_path}")

[youtube] Extracting URL: https://youtube.com/shorts/GAkdsDbmZs8?si=HY3MmqYraE-HPRIf
[youtube] GAkdsDbmZs8: Downloading webpage
[youtube] GAkdsDbmZs8: Downloading tv simply player API JSON
[youtube] GAkdsDbmZs8: Downloading tv client config
[youtube] GAkdsDbmZs8: Downloading tv player API JSON
[info] GAkdsDbmZs8: Downloading 1 format(s): 18
[download] input_video.mp4 has already been downloaded
[download] 100% of    1.04MiB
최종 카운트: 2
오류 프레임 목록: []
저장 완료: output_pose_squat.mp4


In [ ]:
# import cv2
# from ultralytics import YOLO
# import numpy as np
# from moviepy.editor import VideoFileClip
# import os

# # YOLOv8 모델 로드 (Pose)
# model = YOLO('yolov8n-pose.pt')  # 필요한 모델 크기 선택

# # 비디오 파일 열기
# input_video = 'https://youtube.com/shorts/GAkdsDbmZs8?si=HY3MmqYraE-HPRIf'
# cap = cv2.VideoCapture(input_video)

# # 비디오 정보 가져오기
# fps = cap.get(cv2.CAP_PROP_FPS)
# frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# total_duration = total_frames / fps
# frame_count = 0

# # 비디오 저장 설정 (원본 크기 유지)
# fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# out = cv2.VideoWriter('스쿼트_skeleton.mp4', fourcc, fps, (frame_width, frame_height))

# !pip install -q yt-dlp ultralytics opencv-python moviepy Pillow
# CPU 전용 토치가 필요하면 이 줄도 사용:
# !pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

In [ ]:
# 각도 계산 함수
def calculate_angle(a, b, c):
    """3개의 포인트를 받아 각도를 계산"""
    a = np.array(a[:2])  # 첫 번째 점 (x, y)
    b = np.array(b[:2])  # 중앙 점 (관절) (x, y)
    c = np.array(c[:2])  # 세 번째 점 (x, y)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

# 스쿼트 카운트와 상태 추적 변수
count = 0
in_descending_phase = False  # 내려가는 단계인지 추적
in_ascending_phase = False   # 올라오는 단계인지 추적

# 자세 불량 판정을 위한 변수들
bad_posture_detected = False
min_knee_angle_descending = None  # 내려갈 때 무릎 각도 최소값
max_knee_angle_ascending = None   # 올라올 때 무릎 각도 최대값
max_hip_angle_ascending = None    # 올라올 때 엉덩이 각도 최대값 추가

# 신뢰도 임계값 설정 (선택 사항)
confidence_threshold = 0.5

# 오류 시점 기록 리스트
error_frames = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 각도 기본값 설정 (계산되지 않은 경우 대비)
    knee_angle_right = 0
    hip_angle_right = 0
    knee_angle_left = 0
    hip_angle_left = 0

    # YOLOv8을 사용한 포즈 추정
    results = model(frame, verbose=False)

    # 키포인트가 있는지 확인하고 2D 좌표 가져오기
    if hasattr(results[0], 'keypoints'):
        # keypoints를 CPU로 이동시키고 numpy 배열로 변환
        keypoints = results[0].keypoints.xy.cpu().numpy()[0]  # GPU -> CPU, 3차원 -> 2차원으로 변환

        # 허리와 무릎 각도 계산 (필요한 인덱스가 있는지 확인)
        if keypoints.shape[0] >= 17:  # 총 17개의 포인트가 있는지 확인
            # 오른쪽 측면 키포인트
            hip_right = keypoints[11]     # 오른쪽 골반 (인덱스 11)
            knee_right = keypoints[13]    # 오른쪽 무릎 (인덱스 13)
            ankle_right = keypoints[15]   # 오른쪽 발목 (인덱스 15)
            shoulder_right = keypoints[5] # 오른쪽 어깨 (인덱스 5)

            # 왼쪽 측면 키포인트
            hip_left = keypoints[12]      # 왼쪽 골반 (인덱스 12)
            knee_left = keypoints[14]     # 왼쪽 무릎 (인덱스 14)
            ankle_left = keypoints[16]    # 왼쪽 발목 (인덱스 16)
            shoulder_left = keypoints[6]  # 왼쪽 어깨 (인덱스 6)

            # 키포인트가 (x, y, confidence)인지 확인
            has_confidence = keypoints.shape[1] == 3

            if has_confidence:
                # 신뢰도 확인 (선택 사항)
                if (hip_right[2] > confidence_threshold and knee_right[2] > confidence_threshold and ankle_right[2] > confidence_threshold and shoulder_right[2] > confidence_threshold) and \
                   (hip_left[2] > confidence_threshold and knee_left[2] > confidence_threshold and ankle_left[2] > confidence_threshold and shoulder_left[2] > confidence_threshold):

                    # 각도 계산 (오른쪽)
                    knee_angle_right = calculate_angle(hip_right, knee_right, ankle_right)
                    hip_angle_right = calculate_angle(shoulder_right, hip_right, knee_right)

                    # 각도 계산 (왼쪽)
                    knee_angle_left = calculate_angle(hip_left, knee_left, ankle_left)
                    hip_angle_left = calculate_angle(shoulder_left, hip_left, knee_left)
            else:
                # 신뢰도 정보가 없는 경우, 모든 키포인트를 사용
                knee_angle_right = calculate_angle(hip_right, knee_right, ankle_right)
                hip_angle_right = calculate_angle(shoulder_right, hip_right, knee_right)
                knee_angle_left = calculate_angle(hip_left, knee_left, ankle_left)
                hip_angle_left = calculate_angle(shoulder_left, hip_left, knee_left)

            # 좌표를 정수형으로 변환
            shoulder_right_coords = tuple(shoulder_right[:2].astype(int))
            hip_right_coords = tuple(hip_right[:2].astype(int))
            knee_right_coords = tuple(knee_right[:2].astype(int))
            ankle_right_coords = tuple(ankle_right[:2].astype(int))

            shoulder_left_coords = tuple(shoulder_left[:2].astype(int))
            hip_left_coords = tuple(hip_left[:2].astype(int))
            knee_left_coords = tuple(knee_left[:2].astype(int))
            ankle_left_coords = tuple(ankle_left[:2].astype(int))

            # 허리 라인 그리기 (오른쪽과 왼쪽), 노란색
            cv2.line(frame, shoulder_right_coords, hip_right_coords, (0, 255, 255), 2)  # 오른쪽 허리 라인 (노란색)
            cv2.line(frame, shoulder_left_coords, hip_left_coords, (0, 255, 255), 2)    # 왼쪽 허리 라인 (노란색)

            # 허벅지 라인 그리기, 주황색
            cv2.line(frame, hip_right_coords, knee_right_coords, (0, 165, 255), 2)    # 오른쪽 허벅지 라인 (주황색)
            cv2.line(frame, hip_left_coords, knee_left_coords, (0, 165, 255), 2)      # 왼쪽 허벅지 라인 (주황색)

            # 종아리 라인 그리기, 주황색
            cv2.line(frame, knee_right_coords, ankle_right_coords, (0, 165, 255), 2)  # 오른쪽 종아리 라인 (주황색)
            cv2.line(frame, knee_left_coords, ankle_left_coords, (0, 165, 255), 2)    # 왼쪽 종아리 라인 (주황색)

            # 허리와 무릎 각도를 프레임에 그리기
            cv2.putText(frame, f'Hip Angle Right: {hip_angle_right:.2f}', (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(frame, f'Knee Angle Right: {knee_angle_right:.2f}', (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(frame, f'Hip Angle Left: {hip_angle_left:.2f}', (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 165, 255), 2, cv2.LINE_AA)
            cv2.putText(frame, f'Knee Angle Left: {knee_angle_left:.2f}', (10, 140), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 165, 255), 2, cv2.LINE_AA)

            # 오른쪽과 왼쪽의 최소 무릎 각도 선택
            min_knee_angle = min(knee_angle_right, knee_angle_left)
            max_knee_angle = max(knee_angle_right, knee_angle_left)

            # 오른쪽과 왼쪽의 최소 엉덩이 각도 선택
            min_hip_angle = min(hip_angle_right, hip_angle_left)
            max_hip_angle = max(hip_angle_right, hip_angle_left)

            # 올라오는 단계에서 엉덩이 각도 최대값 업데이트
            if in_ascending_phase:
                max_hip_angle_ascending = max(max_hip_angle_ascending or 0, max_hip_angle)

            # 자세 판정을 위한 상태 업데이트
            # 내려가는 단계
            if min_knee_angle < 110:
                if not in_descending_phase:
                    in_descending_phase = True
                    in_ascending_phase = False
                    min_knee_angle_descending = min_knee_angle  # 내려갈 때 무릎 각도 최소값 초기화
                    max_knee_angle_ascending = None
                    max_hip_angle_ascending = None  # 추가된 부분
                else:
                    min_knee_angle_descending = min(min_knee_angle_descending, min_knee_angle)

            # 올라오는 단계
            elif max_knee_angle > 170:
                if in_descending_phase:
                    in_ascending_phase = True
                    max_knee_angle_ascending = max_knee_angle  # 올라올 때 무릎 각도 최대값 초기화
                    max_hip_angle_ascending = max_hip_angle    # 올라올 때 엉덩이 각도 최대값 초기화
                else:
                    # max_knee_angle_ascending이 None이면 max_knee_angle을 할당
                    if max_knee_angle_ascending is None:
                        max_knee_angle_ascending = max_knee_angle
                    else:
                        max_knee_angle_ascending = max(max_knee_angle_ascending, max_knee_angle)

                    # max_hip_angle_ascending이 None이면 max_hip_angle을 할당
                    if max_hip_angle_ascending is None:
                        max_hip_angle_ascending = max_hip_angle
                    else:
                        max_hip_angle_ascending = max(max_hip_angle_ascending, max_hip_angle)

                # 스쿼트 1회 카운트
                if in_ascending_phase and max_knee_angle_ascending > 170 and max_hip_angle_ascending > 170:
                    count += 1
                    in_descending_phase = False
                    in_ascending_phase = False
                    min_knee_angle_descending = None
                    max_knee_angle_ascending = None
                    max_hip_angle_ascending = None
                    # print(f"Count updated: {count}")

            # 충분히 내려가지 않았을 때 (무릎 각도가 120도 이상)
            elif in_descending_phase and min_knee_angle_descending is not None:
                if min_knee_angle_descending > 120:
                    # 자세 불량으로 간주
                    bad_posture_detected = True
                    error_frames.append(frame_count)
                    cv2.putText(frame, '자세불량 탐지: 충분히 내려가지 않음', (10, 180), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                    # print(f"자세불량 frame {frame_count}")
                    in_descending_phase = False
                    min_knee_angle_descending = None
                    max_knee_angle_ascending = None
                    max_hip_angle_ascending = None

            # 충분히 올라오지 않았을 때 (엉덩이 각도가 170도 미만)
            elif in_ascending_phase and max_hip_angle_ascending is not None:
                if max_hip_angle_ascending < 170:
                    # 자세 불량으로 간주
                    bad_posture_detected = True
                    error_frames.append(frame_count)
                    cv2.putText(frame, '자세불량 탐지: 충분히 펴지 않음', (10, 180), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                    # print(f"자세불량 frame {frame_count}")
                    in_ascending_phase = False
                    min_knee_angle_descending = None
                    max_knee_angle_ascending = None
                    max_hip_angle_ascending = None

    # 카운트 텍스트 프레임에 그리기
    cv2.putText(frame, f'Count: {count}', (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # YOLOv8에서 스켈레톤을 그린 프레임과 함께 우리가 그린 정보를 저장
    out.write(frame)  # 'frame'을 저장 (스켈레톤 + 각도와 카운트)

    frame_count += 1

# 비디오 객체 해제
cap.release()
out.release()

print(f"최종 카운트: {count}")
print(f"오류 프레임 목록: {error_frames}")

In [ ]:
# --- 영상 추출 단계 ---

# 오류 프레임이 없으면 추출하지 않음
if error_frames:
    # MoviePy로 비디오 로드
    clip = VideoFileClip(input_video)

    # 출력 디렉토리 설정
    output_dir = '/home/ohseunghwan/AIproject/09healthpose/추출된_영상/'
    os.makedirs(output_dir, exist_ok=True)

    # 오류 시점의 3초 전부터 10초간의 영상 추출
    for idx, error_frame in enumerate(error_frames):
        error_time = error_frame / fps  # 오류 프레임을 초 단위로 변환
        start_time = max(error_time - 3, 0)  # 3초 전, 0보다 작으면 0으로 설정
        end_time = start_time + 10  # 10초간

        # 비디오 길이를 초과하지 않도록 조정
        if end_time > clip.duration:
            end_time = clip.duration

        # 추출된 영상 클립
        subclip = clip.subclip(start_time, end_time)

        # 출력 파일 경로
        output_path = os.path.join(output_dir, f'error_clip_{idx+1}.mp4')

        # 클립 저장
        subclip.write_videofile(output_path, codec="libx264", audio_codec="aac")
        print(f"추출 완료: {output_path}")
else:
    print("오류 프레임이 없습니다. 추출할 영상이 없습니다.")